# Process BenchExec Experiments

This notebook post-processes the results obtained from [Benchexec](https://github.com/sosy-lab/benchexec) tables.

The Benchexec tables contain sets of (repeated) stat columns, one per run set. Each run set can represent a specific solver. Each row is a problem instance, called a _task_ in Benchexec.

This set-up is not very useful to do plotting or stat tables. Instead, we would like to re-shape and have a special column that records the solver, and  so one set of stat columns (with no repetition).

This network will then produce two tables:

1. A flat table of stats, that can be used for further analysis and plotting, with the solver being recorded in a new column.
2. A coverage table per domain and solver, typically reported in papers.

In [1]:
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
import glob
import re

# sys.dont_write_bytecode = True  # prevent creation of .pyc files

CSV_FOLDER = "stats/july24-redo-benchexec/"
NAME_EXPERIMENT = "cfond"

## 1. Flatten Benchexec CSV tables

Collect all CSV benchexec table result files under `CSV_FOLDER` folder.

In [2]:
print(f"CSV files found in folder {CSV_FOLDER}:")
files = glob.glob(os.path.join(CSV_FOLDER, "**", "benchmark-*.csv"), recursive=True)

files += glob.glob(os.path.join(CSV_FOLDER, "**", "results.*.table.csv"), recursive=True)

files

CSV files found in folder stats/july24-redo-benchexec/:


['stats/july24-redo-benchexec/prp-19-07-24/benchmark-prp.2024-07-19_17-59-23.results.prp.FOND.csv',
 'stats/july24-redo-benchexec/paladinus-19-07-24/benchmark-paladinus.2024-07-19_07-50-23.results.paladinus.FOND.csv',
 'stats/july24-redo-benchexec/cfondasp-21-07-24/results.2024-07-29_15-40-06.table.csv',
 'stats/july24-redo-benchexec/fondsat-21-07-24/results.2024-07-29_15-40-20.table.csv']

Next, load all CSV files into a single dataframe. As explained above, each CSV file may include results for many _run sets_, with each having its own (set of) columns.

Each row is the result of a _task_ in the experiment, and every run set has its columns stats for such task. Presumably sets of columns share the same schema.

We then need reshape this structure to have just one set of columns and a new column identifying the run set. So each original row will be expanded into many rows, one per run set.

The first three lines contain header:

1. First line contains the _tool_ used. It starts with `tool` followed by the name of the tool repeated multiple times (to match no. of columns).
2. Second line contains the _runs_ of the experiment. It starts with `run set` and then sets of columns with the name of the runs.
3. Third line contains the _stats_ column names repeated per run set. First column is for name of the task.

In [3]:
RENAME_COLS = {"benchmarks/benchexe/tasks/": "id", "cputime (s)": "cputime", "walltime (s)": "walltime", "memory (MB)": "memory_mb"}

def get_meta_csv(file):
    """Given a benchexec CSV file, extract the runs (e.g., prp, prp_inv) and how many columns per run"""
    with open(file, "r") as f:
        # first line contains the tool used (repeated one per column needed); e.g.,  PP-FOND
        tools_header = f.readline().split()[1:]
        # second line contains the run/solvers used in the experiment (e.g., prp, prp_inv) and starts with "run set" to be ignored
        runs_header = f.readline().split()[2:]

    runs = set(runs_header)

    no_cols = int(len(runs_header) / len(runs))

    return runs, no_cols


dfs = []
for f in files:
    runs, no_cols = get_meta_csv(f)
    print(f"Runs in file {f}: {runs} with {no_cols} stat columns")

    # go over each set of run columns (a csv file may contain many runs, each with the same columns)
    for k, r in enumerate(list(runs)):
        col_idx = [0] + list(range(k*no_cols + 1, k*no_cols + no_cols + 1))
        print(f"\t Extracting run '{r}' in columns: {col_idx}")

        # read the CSV file from line 3+ (line 3 is header)
        df = pd.read_csv(f, delimiter="\t", skiprows=2, usecols=col_idx)
        df.rename(columns=lambda x: x.split('.')[0], inplace=True)

        df.columns.values[0] = "task"
        # df.rename(columns={df.columns[1]: "task"})

        # populate column run with name of run-solver r
        df.insert(1, 'run', r)
        dfs.append(df)

df_csv = pd.concat(dfs).reset_index(drop=True)

df_csv.rename(columns=RENAME_COLS, inplace=True)

print("runs found:", df_csv["run"].unique())
# df.set_index("task", inplace=True)
df_csv

Runs in file stats/july24-redo-benchexec/prp-19-07-24/benchmark-prp.2024-07-19_17-59-23.results.prp.FOND.csv: {'prp.FOND'} with 6 stat columns
	 Extracting run 'prp.FOND' in columns: [0, 1, 2, 3, 4, 5, 6]
Runs in file stats/july24-redo-benchexec/paladinus-19-07-24/benchmark-paladinus.2024-07-19_07-50-23.results.paladinus.FOND.csv: {'paladinus.FOND'} with 6 stat columns
	 Extracting run 'paladinus.FOND' in columns: [0, 1, 2, 3, 4, 5, 6]
Runs in file stats/july24-redo-benchexec/cfondasp-21-07-24/results.2024-07-29_15-40-06.table.csv: {'cfondasp2-fsat.FOND', 'cfondasp1-reg.FOND', 'cfondasp2-reg.FOND', 'cfondasp1-fsat.FOND'} with 6 stat columns
	 Extracting run 'cfondasp2-fsat.FOND' in columns: [0, 1, 2, 3, 4, 5, 6]
	 Extracting run 'cfondasp1-reg.FOND' in columns: [0, 7, 8, 9, 10, 11, 12]
	 Extracting run 'cfondasp2-reg.FOND' in columns: [0, 13, 14, 15, 16, 17, 18]
	 Extracting run 'cfondasp1-fsat.FOND' in columns: [0, 19, 20, 21, 22, 23, 24]
Runs in file stats/july24-redo-benchexec/fonds

,task,run,status,cputime,walltime,memory_mb,planner_time,policy_size
0,acrobatics_01.yml,prp.FOND,true,0.233342,0.235622,10.080256,0.01,4
1,acrobatics_02.yml,prp.FOND,true,10.369209,10.374620,10.199040,10.14,8
2,acrobatics_03.yml,prp.FOND,true,11.289586,11.302472,10.235904,11.05,16
3,acrobatics_04.yml,prp.FOND,true,12.004272,12.007289,10.444800,11.76,32
4,acrobatics_05.yml,prp.FOND,true,17.225556,17.242667,11.210752,16.95,64
...,...,...,...,...,...,...,...,...
4715,zenotravel_11.yml,fondsat-minisat.FOND,TIMEOUT (false),14400.263172,14401.003335,2337.935360,-1.00,-1
4716,zenotravel_12.yml,fondsat-minisat.FOND,TIMEOUT (false),14401.220999,14402.025636,2084.392960,-1.00,-1
4717,zenotravel_13.yml,fondsat-minisat.FOND,TIMEOUT (false),14400.344840,14401.002512,1823.010816,-1.00,-1
4718,zenotravel_14.yml,fondsat-minisat.FOND,TIMEOUT (false),14401.257835,14403.561401,2396.045312,-1.00,-1


In [4]:
# solver/runs found
print("Solvers/run found:", df_csv['run'].unique())

Solvers/run found: ['prp.FOND' 'paladinus.FOND' 'cfondasp2-fsat.FOND' 'cfondasp1-reg.FOND'
 'cfondasp2-reg.FOND' 'cfondasp1-fsat.FOND' 'fondsat-glucose.FOND'
 'fondsat-minisat.FOND']


We next **enrich** the dataframe with the following derived columns:

* domain
* instance
* solver
* solved (boolean)

In [5]:
def get_benchmark_labels(task_name):
    """From the task description name (e.g., acrobatics_01.yml), extract the benchmark labels, like domain, instance"""
    regex = r"(.+)_([0-9]+)\.yml"

    match = re.match(regex, task_name)
    if match:
        # print(match.groups())
        domain = match.group(1)
        instance = match.group(2)
    else:
        print("Problem extracting labels from task name", task_name)
    return domain, instance

df = df_csv.copy()

# 1 - split task name into domain and instances
df["benchmark"] = df.reset_index()["task"].map(get_benchmark_labels).values
df["domain"] = df["benchmark"].str.get(0)
df["instance"] = df["benchmark"].str.get(1)
df.drop(columns=["benchmark"], inplace=True)

# 2 - map status from benchexec to integers status
map_status = {
    "true": 1,
    "false": 0,
    "True": 1,
    "False": 0,
    False: 0,
    True: 1,
    "OUT OF MEMORY (false)": -2,
    "TIMEOUT (false)": -1,
    "TIMEOUT (true)": 1,
}
df["status2"] = df["status"].map(map_status)

missing_mapping = df[df["status2"].isnull()].shape[0]
if missing_mapping > 0:
    missing_status = [x for x in df["status"].unique() if x not in map_status.keys()]
    print(f"WARNING: {missing_mapping} status values not mapped:", missing_status)
    print(df[df["status2"].isnull()])

df["status"] = df["status2"]
df.drop(columns=["status2"], inplace=True)

# 3 - define Boolean column solved to flag if solved or not based on status
df.insert(3, "solved", df["status"].apply(lambda x: True if x == 1 else False))


# 4 - extract solver from run name
map_solver = {
    "prp.FOND": "prp",
    "paladinus.FOND": "paladinus",
    "fondsat-glucose.FOND": "fondsat-glucose",
    "fondsat-minisat.FOND": "fondsat-minisat",
    "cfondasp1-reg.FOND" : "cfondasp1-reg",
    "cfondasp2-reg.FOND" : "cfondasp2-reg",
    "cfondasp1-fsat" : "asp1-fsat",
    "cfondasp2-fsat" : "asp2-fsat",
}
df["solver"] = df["run"].map(map_solver)

# sanity check status
# df.query("status not in [-1,0,-2,1]")
# df.status = df.status.astype(int) # convert to int
# df.loc[df.status == "OUT OF MEMORY (false)"]
# df.loc[df.status == -1]

# df.dtypes

# note that status should be integer; if float it is bc there must be NaN value!
df

,task,run,status,solved,cputime,walltime,memory_mb,planner_time,policy_size,domain,instance,solver
0,acrobatics_01.yml,prp.FOND,1,True,0.233342,0.235622,10.080256,0.01,4,acrobatics,01,prp
1,acrobatics_02.yml,prp.FOND,1,True,10.369209,10.374620,10.199040,10.14,8,acrobatics,02,prp
2,acrobatics_03.yml,prp.FOND,1,True,11.289586,11.302472,10.235904,11.05,16,acrobatics,03,prp
3,acrobatics_04.yml,prp.FOND,1,True,12.004272,12.007289,10.444800,11.76,32,acrobatics,04,prp
4,acrobatics_05.yml,prp.FOND,1,True,17.225556,17.242667,11.210752,16.95,64,acrobatics,05,prp
...,...,...,...,...,...,...,...,...,...,...,...,...
4715,zenotravel_11.yml,fondsat-minisat.FOND,-1,False,14400.263172,14401.003335,2337.935360,-1.00,-1,zenotravel,11,fondsat-minisat
4716,zenotravel_12.yml,fondsat-minisat.FOND,-1,False,14401.220999,14402.025636,2084.392960,-1.00,-1,zenotravel,12,fondsat-minisat
4717,zenotravel_13.yml,fondsat-minisat.FOND,-1,False,14400.344840,14401.002512,1823.010816,-1.00,-1,zenotravel,13,fondsat-minisat
4718,zenotravel_14.yml,fondsat-minisat.FOND,-1,False,14401.257835,14403.561401,2396.045312,-1.00,-1,zenotravel,14,fondsat-minisat


Finally, save all results into a complete CSV file.

In [6]:
df.to_csv(os.path.join(CSV_FOLDER, f"{NAME_EXPERIMENT}_benchexec_stats.csv"), index=False)

## 2. Coverage Analysis and Table

We now generate **coverage** tables, as they often appear in papers. Basically, we compute the following per domain and solver-run:

- **Coverage:** % of solved instances solved by the solver-run; and
- **Stat metrics:** mean on time, memory usage, and policy size.

In [7]:
print(df.shape)
df.head()

(4720, 12)


,task,run,status,solved,cputime,walltime,memory_mb,planner_time,policy_size,domain,instance,solver
0,acrobatics_01.yml,prp.FOND,1,True,0.233342,0.235622,10.080256,0.01,4,acrobatics,01,prp
1,acrobatics_02.yml,prp.FOND,1,True,10.369209,10.374620,10.199040,10.14,8,acrobatics,02,prp
2,acrobatics_03.yml,prp.FOND,1,True,11.289586,11.302472,10.235904,11.05,16,acrobatics,03,prp
3,acrobatics_04.yml,prp.FOND,1,True,12.004272,12.007289,10.444800,11.76,32,acrobatics,04,prp
4,acrobatics_05.yml,prp.FOND,1,True,17.225556,17.242667,11.210752,16.95,64,acrobatics,05,prp


Calculate % ratio per set/domain/sub_domain/run-solver.

In [11]:
df_grouped = df.groupby(["domain", "solver"])

#   df_grouped.sum()[["solved"]] = sum all the True instances (sum over bool = number of True)
#   df_grouped.count()[["solved"]] = number of rows in solved column (includes True and False values)
df_coverage = (
    df_grouped.sum(numeric_only=True)[["solved"]] / df_grouped.count()[["solved"]]
)
df_coverage

solved
domain     solver                   
acrobatics cfondasp1-reg    0.500000
           cfondasp2-reg    0.000000
           fondsat-glucose  0.375000
           fondsat-minisat  0.500000
           paladinus        1.000000
...                              ...
zenotravel cfondasp2-reg    0.000000
           fondsat-glucose  0.333333
           fondsat-minisat  0.333333
           paladinus        0.333333
           prp              1.000000

[102 rows x 1 columns]

Calculate mean metric (for CPU time, memory, and policy size) across the solved instances.

In [12]:
columns = ["domain", "solver", "cputime", "memory_mb", "policy_size"]
df_solved = df.query("solved == True")[columns]

df_solved_grouped = df_solved.groupby(["domain", "solver"])
df_metrics = df_solved_grouped.mean()
df_metrics

cputime   memory_mb  policy_size
domain     solver                                              
acrobatics cfondasp1-reg    688.996671   80.991232    15.000000
           fondsat-glucose    7.889886   23.134208     9.333333
           fondsat-minisat  178.673675   82.584576    15.000000
           paladinus         11.176993   55.968768   126.500000
           prp               13.011640   12.451840   127.500000
...                                ...         ...          ...
zenotravel cfondasp1-reg    940.044956  179.512320    15.500000
           fondsat-glucose  406.648567  397.922304    13.000000
           fondsat-minisat  170.958383  466.844058    13.000000
           paladinus         16.789707  368.121446    15.200000
           prp                3.753867   66.439305    54.600000

[85 rows x 3 columns]

Put together **Coverage** and **Metrics** tables.

In [ ]:
column_names = {
    "solved": "cov",
    "cputime": "time",
    "memory_mb": "mem",
    "policy_size": "size",
}

df_stats = df_coverage.join(df_metrics, how="inner")
df_stats.rename(columns=column_names, inplace=True)

df_stats = df_stats.reset_index()

df_stats

In [ ]:
df_stats_pivot = df_stats.pivot(
    index=["domain"],
    values=["cov", "time", "mem", "size"],
    columns="solver",
)
df_stats_pivot.reset_index(
    inplace=True
)  # unfold multi-index into columns (create integer index)
df_stats_pivot.columns = [
    "_".join(tup).rstrip("_") for tup in df_stats_pivot.columns.values
]

# flat index, but multi-column: 1. coverage / time / policy size and 2. each solver/run
df_stats_pivot = df_stats_pivot.round(2)

df_stats_pivot

Save it to the file, this can be used in the paper.

In [ ]:
df_stats_pivot.to_csv(os.path.join(CSV_FOLDER, f"{NAME_EXPERIMENT}_coverage_table.csv"), index=False)